## Setup

In [1]:
!pip install -q openai rich ujson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [1]:
import openai
import os
import re
from rich.console import Console
import re
import getpass
import json
from jinja2 import Template
import sys
from tqdm.auto import tqdm
from google.colab import files
import ujson

api_key = getpass.getpass("Enter the OpenAI API Key: ")
assert api_key.startswith("sk-"), 'OpenAI API Keys begin with "sk-".'
openai.api_key = api_key

Enter the OpenAI API Key: ··········


In [2]:
card_extract_pattern = r'(\{"name":.*rarity.*\})'
def color_rarity(value, to_notebook=True):
    value = value.lower()
    # Can only display colors if printing to the notebook
    if to_notebook:
        if "mythic" in value:
            value = f"[bright_magenta]{value}[/bright_magenta]"
        elif "rare" in value:
            value = f"[bright_blue]{value}[/bright_blue]"
        elif "uncommon" in value:
            value = f"[bright_green]{value}[/bright_green]"
    return value

TEMPLATE = Template(
    """{{ c.name }}{% if c.manaCost %}  {{ c.manaCost }}{% endif %}
{{ c.type }}{% if c.text %}
{{ c.text }}{% endif %}{% if c.flavorText %}
{{ c.flavorText }}{% endif %}{% if c.pt %}
{{ c.pt }}{% elif c.loyalty %}
Loyalty: {{ c.loyalty }}{% endif %}
{{ c.rarity }}"""
)


def render_card(card_dict):
    return TEMPLATE.render(c=card_dict)

In [52]:
system = """You are an assistant who works as a Magic: The Gathering card designer. Create cards that are in the following card schema and JSON format. OUTPUT MUST FOLLOW THIS CARD SCHEMA AND JSON FORMAT. Always escape double-quotes when necessary. The output must also follow the Magic "color pie".

{"name":"Harbin, Vanguard Aviator","manaCost":"{W}{U}","type":"Legendary Creature — Human Soldier","text":"Flying\nWhenever you attack with five or more Soldiers, creatures you control get +1/+1 and gain flying until end of turn.","flavorText":"\"Yotia is my birthright, father. Let me fight for it.\"","pt":"3/2","rarity":"rare"}

After outputting a card, respond with "<|DONE|>"."""

def generate_magic_cards(prompt):
    r = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        stop="<|DONE|>",
        max_tokens=1000,  # sanity limit
        temperature=0.8,  # increasing temp higher than 0.8 may cause non-JSON output
    )
    return r["choices"][0]["message"]["content"]


## Generate the Card!

Some example inputs:

- Create a five-color Mythic Rare Artifact Magic card.
- Create a Planeswalker Magic card with the subtype "Bob".
- Create a mechanically unique Land Magic card.
- Create a Magic card with "What's updog?" flavorText.
- Create a Magic card named "Steve Jobs" with a mana cost of atleast ten.
- Create a multicolor Magic card based on the War of 1812.
- Create a common Magic card based on Final Fantasy VII.
- Create a mechanically unique Magic card based on a crossover between the War of 1812 and Final Fantasy VII.

The cell will also output the raw ChatGPT output for posterity / debugging if output has unexpected issues.

In [90]:
from ujson import JSONDecodeError
num_repeat_to_file = 10
output_width = 60
success = True

prompt = "Create a mechanically unique Magic card based on a crossover between the War of 1812 and Final Fantasy VII." #@param {type:"string"}
to_file = False  # not quite working yet

outputs = generate_magic_cards(prompt)

console = Console(width=60)
if to_file:
    outputs = []
    for _ in tqdm(range(num_repeat_to_file)):
        temp_outputs = generate_magic_cards(prompt)
        for output in temp_outputs:
            outputs.append(output)
    outputs = "\n\n".join(outputs)
else:
    outputs = generate_magic_cards(prompt)

cards = re.findall(card_extract_pattern, outputs, re.S)
if len(cards) == 0:
    console.print(f"[bright_red]<No cards generated! 😭>\nModel output: {outputs}[/bright_red]", highlight=False)
    success = False

processed_cards = []

for card in cards:
    try:
        card = ujson.loads(card)
        card["name"] = f"[bold]{card['name']}[/bold]"
        if "flavorText" in card:
            card["flavorText"] = f"[italic]{card['flavorText']}[/italic]"
        card["rarity"] = color_rarity(card["rarity"])
        if "power" in card and "toughness" in card:
            card["pt"] = f'{card["power"]}/{card["toughness"]}'
        processed_cards.append(render_card(card))
    except JSONDecodeError as e:
        console.print(f"[bright_red]<JSON Parsing Failed! 😭>\nModel output: {outputs}[/bright_red]", highlight=False)
        success = False
        continue

if to_file:
    file_path = "cards.txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(("\n" + "─" * output_width + "\n").join(processed_cards))
    if "google.colab" in sys.modules:
        files.download(file_path)
elif success:
    console = Console(width=output_width)
    console.print(("\n" + "─" * output_width + "\n").join(processed_cards), highlight=False)
    console.print("─" * output_width, highlight=False)
    console.print(f"ChatGPT Output:\n{outputs}", highlight=False)

Sephiroth, Burning Invader  {2}{B}{B}{R}{R}
Legendary Creature — Human Warrior
First strike, haste
Whenever Sephiroth, Burning Invader deals combat damage to a
player, you may sacrifice another creature. If you do, 
Sephiroth deals damage equal to the sacrificed creature's 
power to any target.
Whenever an artifact is put into a graveyard from the 
battlefield, put a +1/+1 counter on Sephiroth.
The British may have burned the White House, but Sephiroth 
will burn the world.
4/3
mythic rare

────────────────────────────────────────────────────────────

ChatGPT Output:
{"name":"Sephiroth, Burning 
Invader","manaCost":"{2}{B}{B}{R}{R}","type":"Legendary 
Creature — Human Warrior","text":"First strike, haste
Whenever Sephiroth, Burning Invader deals combat damage to a
player, you may sacrifice another creature. If you do, 
Sephiroth deals damage equal to the sacrificed creature's 
power to any target.
Whenever an artifact is put into a graveyard from the 
battlefield, put a +1/+1 counter on 
Sephiroth.","flavorText":"The British may have burned the 
White House, but Sephiroth will burn the 
world.","pt":"4/3","rarity":"mythic rare"}